In [1]:
#!pip install --upgrade numpy
#!pip install Cython
#!pip install nemo_toolkit\['all'\]

In [18]:
import nemo.collections.tts as nemo_tts
import pydub 
import librosa
import soundfile
import torch

In [14]:
#tts = nemo_tts.models.FastPitchModel.from_pretrained("tts_en_fastpitch_multispeaker").cpu().eval()
tts = nemo_tts.models.FastPitchModel.from_pretrained("tts_en_fastpitch_multispeaker").cpu().eval()
#vocoder = nemo_tts.models.WaveGlowModel.from_pretrained("tts_waveglow_88m").cpu().eval()
vocoder = nemo_tts.models.UnivNetModel.from_pretrained("tts_en_libritts_univnet").cpu().eval()

[NeMo I 2022-12-12 12:12:28 cloud:56] Found existing object /home/nikita/.cache/torch/NeMo/NeMo_1.13.0/tts_en_fastpitch_multispeaker/b96451578e2b9a39d62cfc642bc5ffcf/tts_en_fastpitch_multispeaker.nemo.
[NeMo I 2022-12-12 12:12:28 cloud:62] Re-using file from: /home/nikita/.cache/torch/NeMo/NeMo_1.13.0/tts_en_fastpitch_multispeaker/b96451578e2b9a39d62cfc642bc5ffcf/tts_en_fastpitch_multispeaker.nemo
[NeMo I 2022-12-12 12:12:28 common:911] Instantiating model from pre-trained checkpoint
[NeMo I 2022-12-12 12:12:30 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-12-12 12:12:57 modules:94] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-12-12 12:12:57 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /ws/HiFiTTS/manifest_train_ngc.json
      sample_rate: 44100
      sup_data_path: /raid/sup_data_path_44100_new
      sup_data_types:
      - align_prior_matrix
      - pitch
      - speaker_id
      n_fft: 2048
      win_length: 2048
      hop_length: 512
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: null
      max_duration: 20
      mi

[NeMo I 2022-12-12 12:12:57 features:225] PADDING: 1
[NeMo I 2022-12-12 12:12:58 save_restore_connector:243] Model FastPitchModel was successfully restored from /home/nikita/.cache/torch/NeMo/NeMo_1.13.0/tts_en_fastpitch_multispeaker/b96451578e2b9a39d62cfc642bc5ffcf/tts_en_fastpitch_multispeaker.nemo.
[NeMo I 2022-12-12 12:12:58 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/tts_en_libritts_univnet/versions/1.7.0/files/tts_en_libritts_multispeaker_univnet.nemo to /home/nikita/.cache/torch/NeMo/NeMo_1.13.0/tts_en_libritts_multispeaker_univnet/74805fd10315e0a1a4bfe0f7b55ee1b5/tts_en_libritts_multispeaker_univnet.nemo
[NeMo I 2022-12-12 12:13:36 common:911] Instantiating model from pre-trained checkpoint


[NeMo W 2022-12-12 12:13:36 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.AudioDataset
      manifest_filepath: /home/tkdrlf9202/Datasets/LibriTTS-22k/train-all.json
      max_duration: null
      min_duration: 0.75
      n_segments: 16384
      trim: false
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 8
      num_workers: 4
    
[NeMo W 2022-12-12 12:13:36 modelPT:149] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.AudioDataset
      manifest_filepath: /home/tkdrlf9202/Datasets/L

[NeMo I 2022-12-12 12:13:36 features:225] PADDING: 0
[NeMo I 2022-12-12 12:13:36 features:233] STFT using exact pad
[NeMo I 2022-12-12 12:13:36 features:225] PADDING: 0
[NeMo I 2022-12-12 12:13:36 features:233] STFT using exact pad
[NeMo I 2022-12-12 12:13:37 save_restore_connector:243] Model UnivNetModel was successfully restored from /home/nikita/.cache/torch/NeMo/NeMo_1.13.0/tts_en_libritts_multispeaker_univnet/74805fd10315e0a1a4bfe0f7b55ee1b5/tts_en_libritts_multispeaker_univnet.nemo.


In [21]:
def say(word = "gingival"):
    tts_parsed_input = tts.parse(word)
    with torch.no_grad():
        specs = tts.generate_spectrogram(tokens=tts_parsed_input)
        audio = vocoder.convert_spectrogram_to_audio(spec=specs)
    return audio

In [39]:
text = "18"
audio = say(text)

In [40]:
soundfile.write(f"generated/{text}.wav", audio.cpu().numpy()[0], samplerate=22050)

In [41]:
pydub.AudioSegment.from_file(f"generated/{text}.wav")

In [34]:
pydub.AudioSegment.from_file(f"test/fe8086cd8ac643cd99aaeffce48dfa8b_245570ms-248350ms.wav")
